In [369]:
import pandas as pd
import numpy as np
import time
import datetime
import json
import os
import requests
from bs4 import BeautifulSoup as bs
import re
from urllib.parse import unquote
import PyPDF2 as p2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### read csv master file with Project Name and link to ESA folder(s) that contain the ESA files
### 85 rows, because some projects have multiple folder (which contain the ESA documents)  

In [370]:
path_to_master_file = 'F:/Environmental Baseline Data/Version 4 - Final/Support files/list_of_Section_52_projects.csv'
master = pd.read_csv(path_to_master_file, encoding='ISO-8859-1')
master['folder_link'] = master['link to folder with ESA'].str.replace('/View/', '/LoadResult/')
print('shape of csv:', master.shape)
print('numer of unique project:', master['Application title'].nunique())

shape of csv: (85, 7)
numer of unique project: 37


### get all RegDocs files

In [371]:
%%time

all_esa = []
list_applications = master['Application title'].tolist()
list_downloads = master['folder_link'] 

for a, b in zip(list_downloads, list_applications):
    cookies = {'RDI-NumberOfRecords': '200'}
    r = requests.get(a, cookies = cookies)
    soup = bs(r.text, 'lxml')
    tds = soup.find_all('td', {'class' : 'nopadding break-anywhere'})
    file_name = [x.text for x in soup.find_all('a', {'class' : 'row-icon'})]
    hrefs = [a['href'] for a in soup.find_all('a', {'class' : 'row-icon'})]
    df = pd.DataFrame({'file_name' : file_name, 'hrefs' : hrefs, 'application_name' : b, 'esa_folder_link' : a})
    all_esa.append(df)

df_esa = pd.concat(all_esa, axis = 0)
df_esa.shape
df_esa.columns
# df_esa.head()

Wall time: 20.5 s


Index(['file_name', 'hrefs', 'application_name', 'esa_folder_link'], dtype='object')

In [373]:
# create download links by going to each folder and identify the ESA files (using regular expressions) 
df_esa = pd.concat(all_esa, axis = 0)
regex_esa = 'COVER|NV|EPP|EIS|ESA|nvironment|Horn|Vantage|7212_App|ocio|EIA|App 13|V11|V14|V15|V16|V17|V18|V19|V20|V21|V22|V23|V24|V25|A1X1|A1C3|A3S1|A3S2|Goldboro'
df_esa = df_esa[df_esa['file_name'].str.contains(regex_esa)]
df_esa['DocumentID'] = df_esa['file_name'].str.extract('([a-zA-Z]\d[a-zA-Z]\d[a-zA-Z]\d)')
df_esa['DataID'] = df_esa['hrefs'].str.replace('/REGDOCS/File/Download/', '')
df_esa['esa_download_link'] = df_esa['hrefs'].str.replace('/REGDOCS/', 'https://apps.cer-rec.gc.ca/REGDOCS/')
df_esa = df_esa.drop('hrefs', axis = 1)
df_esa['DataID_pdf'] = df_esa['DataID'] + '.pdf'

# remove receipts
df_esa = df_esa[~df_esa['file_name'].str.contains('eceipt')]

In [374]:
#delete these files
# B-1S  - Section 12 - Environmental and Socio-Economic Matters  - A2F4K6
# B-1T - Horn_River_Komie_North_Overview_Map_2011-09-29  - A2F4K7 
# B-1-g -- A0U3G2 - Environmental and Socio-Economic Assessment - Volume 2 of 2 (Paper Only)
# B1-15 - 12.0 Environmental and Socio-Economic - A2A6Q1



In [375]:
# %%time
# # downloads all files
# # ****************************************************************** 1 hour execution 

# download_link = df_esa['esa_download_link'].tolist()
# DataID = df_esa['DataID'].tolist()
# save_folder = 'F:/Environmental Baseline Data/Version 4 - Final/PDF/'

# for x, y in zip(download_link, DataID):
#     try:
#         download_url = 'http://docs2.cer-rec.gc.ca/ll-eng/llisapi.dll?func=ll&objId=' + y + '&objaction=download&viewType=1'
#         r = requests.get(download_url)
#         full_name = os.path.join(save_folder, (y +'.pdf')) 
#         with open(full_name, 'wb') as file:
#             file.write(r.content) 
#     except:
#         print(x, y)


In [376]:
# download all files per hearing
# *******************************************************************
# per_project = df_esa[df_esa['application_name'] == '2010-02-19 - Application for the Horn River Project (GH-2-2010)']

# download_link = per_project['esa_download_link'].tolist()
# DataID = per_project['DataID'].tolist()
# save_folder = 'F:/Environmental Baseline Data/Version 4 - Final/PDF/'
# len(download_link)
# len(DataID)

# for x, y in zip(download_link, DataID):
#     download_url = 'http://docs2.cer-rec.gc.ca/ll-eng/llisapi.dll?func=ll&objId=' + y + '&objaction=download&viewType=1'
#     r = requests.get(download_url)
#     full_name = os.path.join(save_folder, (y +'.pdf')) 
#     with open(full_name, 'wb') as file:
#         file.write(r.content) 

## load GIS info and merge

In [378]:
gis_df = pd.read_excel('F:/Environmental Baseline Data/Version 4 - Final/Indices/Index 1 - List of Major Projects with ESAs.xlsx')
gis_df.columns
df_esa_with_gis = df_esa.merge(gis_df, on = 'application_name', how = 'left')

Index(['Section', 'application_name', 'Application filing date',
       'link to folder with ESA', 'Hearing order', 'Folder link',
       'Company Name', 'Consultant Name', 'Status in GIS', 'Column1',
       'RTS English Title = Project name', 'Regulatory Instruments Issued',
       'Pipeline Named in GIS', 'Application title short',
       'Link to Application', 'Link to Decision', 'Location'],
      dtype='object')

## load component info and merge

In [380]:
esa_component_path = 'F:/Environmental Baseline Data/Version 4 - Final/Support files/Appendices mapped with files per Project.xlsx'
esa_component_names = pd.read_excel(esa_component_path, sheet_name = 'new')

esa_component_names.columns

Index(['Application title short', 'Component', 'Component PDF Name',
       'file_name', 'Component Index'],
      dtype='object')

In [381]:
#load component and merge 

df_esa_with_gis_components = df_esa_with_gis.merge(esa_component_names, on = 'file_name', how = 'left')
df_esa_with_gis_components = df_esa_with_gis_components.drop('Application title short_y', axis = 1)
df_esa_with_gis_components = df_esa_with_gis_components.rename(columns = {'Application title short_x' : 'Application title short'})
df_esa_with_gis_components = df_esa_with_gis_components.drop('Location', axis = 1)
df_esa_with_gis_components.head(2)
df_esa_with_gis_components.shape
# df_esa_with_gis_components.columns
# print(df_esa_with_gis_components['Application title short'].unique())
# df_esa_with_gis_components.tail(3)
# df_esa_with_gis_components.to_csv('df_esa_with_gis_components.csv')
# test = df_esa_with_gis_components[df_esa_with_gis_components['Application title short'] == 'Application for Trans Mountain Expansion Project']
# len(test)

,file_name,application_name,esa_folder_link,DocumentID,DataID,esa_download_link,DataID_pdf,Section,Application filing date,link to folder with ESA,...,Column1,RTS English Title = Project name,Regulatory Instruments Issued,Pipeline Named in GIS,Application title short,Link to Application,Link to Decision,Component,Component PDF Name,Component Index
0,A0H8C0 - 13.0 EIA - Section 13.1 to 13.6,2003-03-17 Application to Construct and Operat...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A0H8C0,268706,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,268706.pdf,Large Projects (over 40 km),2003-03-17,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,...,Operating,Ekwan Pipeline Application/Horn River Pipeline...,GC-108,Ekwan Pipeline,Application to Construct and Operate Ekwan Pip...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,ESA,"Section 13.1: Introduction, Section 13.1: Proj...",1.0
1,A0H8C1 - 13.0 EIA - Section 13.7 Wildlife Part 1,2003-03-17 Application to Construct and Operat...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A0H8C1,268709,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,268709.pdf,Large Projects (over 40 km),2003-03-17,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,...,Operating,Ekwan Pipeline Application/Horn River Pipeline...,GC-108,Ekwan Pipeline,Application to Construct and Operate Ekwan Pip...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,ESA,Section 13.7: Wildlife and Wildlife Habitat,2.0


(1905, 25)

## load short_names and merge

In [382]:
short_names = pd.read_csv('F:/Environmental Baseline Data/Version 4 - Final/Support files/short_names_update.csv')
# short_names.head()
df_esa_with_gis_components_short_names = df_esa_with_gis_components.merge(short_names, on = 'application_name', how = 'left')
df_esa_with_gis_components_short_names.shape
df_esa_with_gis_components_short_names.columns

test = df_esa_with_gis_components_short_names[df_esa_with_gis_components_short_names['Application title short'] == 'Application for Trans Mountain Expansion Project']
len(test)

(1905, 29)

Index(['file_name', 'application_name', 'esa_folder_link', 'DocumentID',
       'DataID', 'esa_download_link', 'DataID_pdf', 'Section',
       'Application filing date', 'link to folder with ESA', 'Hearing order',
       'Folder link', 'Company Name', 'Consultant Name', 'Status in GIS',
       'Column1', 'RTS English Title = Project name',
       'Regulatory Instruments Issued', 'Pipeline Named in GIS',
       'Application title short', 'Link to Application', 'Link to Decision',
       'Component', 'Component PDF Name', 'Component Index', 'short_name',
       'Location', 'Commodity', 'Download folder name'],
      dtype='object')

538

## Load table title and merge with regdocs metadata

In [383]:
#this section loads text files that contain table titles
json_path = 'F:/Environmental Baseline Data/Version 4 - Final/CSV_final_JSON'

path = json_path
paths = os.listdir(path)
all_paths = [path +  '/' + str(x) for x in paths]
len(all_paths)

df_ = pd.DataFrame()

for path in all_paths:
    #load json file; save dictionary in variable d
    with open(path) as f: 
        d = json.load(f)
    #ignore dictionary of empty
    if not d:
        continue
    else:
        my_df = pd.concat({k: pd.Series(v) for k, v in d.items()})
        frames = [df_,my_df]
        df_ = pd.concat(frames)
        
#save a copy
df_tmp = df_
#arrange columns and indices
df_.reset_index(level=0, inplace=True)
df_ = df_.rename(columns={'index': 'file_page_table', 0:'table_name'})

#remove rows where title and filename are the same
for index, row in df_.iterrows():
    lst = row['table_name']
    lst_item2 = lst[1].replace('.csv','')
    lst_item3 = lst[2]
    if lst_item2 == lst_item3:
        df_.drop(index , inplace=True)

1900

In [384]:
%%time 
#Wall time: 1.53 s
# Some clean up of the data, thus using regular expressions
# This section uses Janna optimized version of Veibudhs code; does not match Vibudh's code identically.

df_table_titles = df_.copy()
df_table_titles['table_name'] = df_table_titles['table_name'].astype(str)
df_table_titles['table_name_short'] = [x[5:] for x in df_table_titles['table_name']]
df_table_titles['csv_name'] = df_table_titles['table_name'].str.extract(r'(\S+.csv)')
df_table_titles['table_name_short'] = df_table_titles['table_name_short'].str.replace("\d+\_\d+\_\d+.csv\'\,", '')
df_table_titles['table_name_short'] = df_table_titles['table_name_short'].str.replace('xa0', ' ')
df_table_titles['table_name_short'] = df_table_titles['table_name_short'].str.replace('\\', '')
df_table_titles['table_name_short'] = df_table_titles['table_name_short'].str.replace("\(Cont.D\).+", '')
df_table_titles['table_name_short'] = df_table_titles['table_name_short'].str.replace("(Cont'D)", '')
df_table_titles['table_name_short'] = df_table_titles['table_name_short'].str.replace("\_\d+\_\d+", '')
df_table_titles['table_name_short'] = df_table_titles['table_name_short'].str.replace("'T", 'T')
df_table_titles['table_name_short'] = df_table_titles['table_name_short'].str.replace("\(Continued\)", '') 
df_table_titles['table_name_short'] = df_table_titles['table_name_short'].str.replace("\_\d']", '')
df_table_titles['table_name_short'] = df_table_titles['table_name_short'].str.replace('\(\)', '')
df_table_titles['table_name_short'] = df_table_titles['table_name_short'].str.replace('This Table.+', '')
df_table_titles['table_name_short'] = df_table_titles['table_name_short'].str.replace('\'\d{5,}_', '') 
df_table_titles['table_name_short'] = df_table_titles['table_name_short'].str.replace('\']', '') 
df_table_titles['table_name_short'] = df_table_titles['table_name_short'].str.replace('1 Biophysical Or.+', '') 
df_table_titles['table_name_short'] = df_table_titles['table_name_short'].str.replace('\d{1,2}.Pred.+', '')
df_table_titles['table_name_short'] = df_table_titles['table_name_short'].str.replace('\"\d+_', '')
df_table_titles['table_name_short'] = df_table_titles['table_name_short'].str.replace('\"]', '')
df_table_titles['table_name_short'] = df_table_titles['table_name_short'].str.replace('\s\sMitigation\s\s.+', '')
df_table_titles['table_name_short'] = df_table_titles['table_name_short'].str.replace('\s\sKey\s\sMagnitude\s\s.+', '')
df_table_titles['table_name_short'] = df_table_titles['table_name_short'].str.replace('Mitigation\s1.+|Follow-Up And+| \
                                                                                      ..Monitoring..+|Key Magnitude.+' , '')

df_table_titles['table_name_short'] = df_table_titles['table_name_short'].str.strip()

# remove this b/c not a table; 
df_table_titles = df_table_titles[~df_table_titles['table_name_short'].str.contains('Provides A Summary Of ')].reset_index(drop = True)

df_table_titles = df_table_titles.drop(['file_page_table', 'table_name'], axis = 1)
df_table_titles['csv_name'] = df_table_titles['csv_name'].str.replace("'", '')
df_table_titles[['DataID', 'Page Number','Table Number']] = df_table_titles['csv_name'].str.split('_', expand = True)
df_table_titles['Table Number'] = df_table_titles['Table Number'].str.replace('.csv', '')

df_table_titles.head()   
# df_table_titles.columns
# df_table_titles.tail() 
# df_table_titles.to_csv('test.csv', index = False, encoding = 'utf_8_sig')

Wall time: 1.65 s


,table_name_short,csv_name,DataID,Page Number,Table Number
0,Table 1 Hydrometric Data From Discharge Statio...,1059614_8_1.csv,1059614,8,1
1,Table 2 Fish Species That May Occur In The Upp...,1059614_9_1.csv,1059614,9,1
2,Table 3 Summary Of Aquatics Field Work And Abo...,1059614_14_1.csv,1059614,14,1
3,Table 4 Summary Of Watercourse Crossings Along...,1059614_17_1.csv,1059614,17,1
4,Table 4 Summary Of Watercourse Crossings Along...,1059614_18_1.csv,1059614,18,1


In [386]:
#create list of csv links
csv_list = df_table_titles['csv_name'].tolist()
csv_list_path = ['F:/Environmental Baseline Data/Version 4 - Final/CSV_final/' + x for x in csv_list]
csv_list_path[0:4]

['F:/Environmental Baseline Data/Version 4 - Final/CSV_final/1059614_8_1.csv',
 'F:/Environmental Baseline Data/Version 4 - Final/CSV_final/1059614_9_1.csv',
 'F:/Environmental Baseline Data/Version 4 - Final/CSV_final/1059614_14_1.csv',
 'F:/Environmental Baseline Data/Version 4 - Final/CSV_final/1059614_17_1.csv']

In [387]:
%%time 
#Wall time: 3min 6s
#go through each CSV file and extract shape, and text; save to dict_pandas

exceptions_files = []
dict_pandas = []

for x in csv_list_path:
    try:
#         print(x)
        df = pd.read_csv(x)
        shape = df.shape
        first_row = str(list(df.iloc[:,0]))
        columns = list(df.columns)       
        onestring = str(df.values) #df = df.astype(str)
        dictionary = {'csv_path' : x, 'shape' : (shape), 'columns_index' : columns, 'row_index' : first_row, 'text_df_all' : onestring }
        dict_pandas.append(dictionary) 
    except:
        exceptions_files.append(x)

len(dict_pandas)

Wall time: 3min 10s


27058

In [388]:
%%time 
#Wall time: 1.39 s
#
#read dictionary
df_csv_all = pd.DataFrame(dict_pandas)

df_csv_all['csv_name'] = df_csv_all['csv_path'].str.split('/').str[4]
df_csv_all['contains_cids'] = np.where(df_csv_all['text_df_all'].str.contains("cid:", case=False, na=False), 'contains cids', 'no cids')          
df_csv_all['contains_cids'].value_counts()
df_csv_all.head(2)

# test = df_csv_all[df_csv_all['Application title short'] == 'Application for Trans Mountain Expansion Project']
# len(test)

Wall time: 1.39 s


,csv_path,shape,columns_index,row_index,text_df_all,csv_name,contains_cids
0,F:/Environmental Baseline Data/Version 4 - Fin...,"(6, 6)","[Name, Station Name, Station No., Years Statio...","['Pine River', 'Moberly River', 'Peace River',...",[['Pine River' 'Pine River at East Pine 07FB00...,1059614_8_1.csv,no cids
1,F:/Environmental Baseline Data/Version 4 - Fin...,"(36, 5)","[Common Name <s>1</s>, Scientific Name<s>1</s>...","['SPORTFISH', 'Arctic grayling', 'rainbow trou...",[['SPORTFISH' nan nan nan nan]\n ['Arctic gray...,1059614_9_1.csv,no cids


In [389]:
#merge the two dataframes with tables data
tabels_csv_df = df_table_titles.merge(df_csv_all, on = 'csv_name', how = 'left')
tabels_csv_df = tabels_csv_df.rename(columns={"table_name_short": "Title"})
tabels_csv_df.shape
tabels_csv_df.head(2)

(27058, 11)

,Title,csv_name,DataID,Page Number,Table Number,csv_path,shape,columns_index,row_index,text_df_all,contains_cids
0,Table 1 Hydrometric Data From Discharge Statio...,1059614_8_1.csv,1059614,8,1,F:/Environmental Baseline Data/Version 4 - Fin...,"(6, 6)","[Name, Station Name, Station No., Years Statio...","['Pine River', 'Moberly River', 'Peace River',...",[['Pine River' 'Pine River at East Pine 07FB00...,no cids
1,Table 2 Fish Species That May Occur In The Upp...,1059614_9_1.csv,1059614,9,1,F:/Environmental Baseline Data/Version 4 - Fin...,"(36, 5)","[Common Name <s>1</s>, Scientific Name<s>1</s>...","['SPORTFISH', 'Arctic grayling', 'rainbow trou...",[['SPORTFISH' nan nan nan nan]\n ['Arctic gray...,no cids


In [391]:
#merge table data with regdocs metadata
tabels_csv_df_cer_meta = tabels_csv_df.merge(df_esa_with_gis_components_short_names, on = 'DataID', how = 'left')

In [392]:
#create download strings 
tabels_csv_df_cer_meta['csv_download_link'] = 'http://dweb5/esa-ees/' \
                                + tabels_csv_df_cer_meta['Download folder name'] \
                                + '/' + tabels_csv_df_cer_meta['csv_name']

### Save Index 3 (no cids)

In [393]:
# save data as Index 3 V3 CSV2 LITE.csv
tabels_csv_df_cer_meta_no_cids = tabels_csv_df_cer_meta.drop(['text_df_all', 'columns_index', 'row_index', 'contains_cids'], axis = 1)
tabels_csv_df_cer_meta_no_cids['Content type'] = 'Table'
tabels_csv_df_cer_meta_no_cids.shape
# tabels_csv_df_cer_meta_no_cids.columns
tabels_csv_df_cer_meta_no_cids.to_csv('F:/Environmental Baseline Data/Version 4 - Final/Indices/Index 3 V3 CSV2 LITE.csv', encoding = 'utf_8_sig')

(27058, 37)

### Save Index with cids (these csv files need minor test edits; extraction issue)

In [394]:
# tabels_csv_df_cer_meta.columns
# tabels_csv_df_cer_meta.shape
# tabels_csv_df_cer_meta.to_csv('F:/Environmental Baseline Data/Version 4 - Final/Indices/Index3_full_cid_V2s.csv', encoding = 'utf_8_sig')

# tabels_csv_df_cer_metas_cids_only = tabels_csv_df_cer_meta[tabels_csv_df_cer_meta['contains_cids'] == 'contains cids']
# tabels_csv_df_cer_metas_cids_only.to_csv('F:/Environmental Baseline Data/Version 4 - Final/Indices/Index3_full_cids_only_V2.csv', encoding = 'utf_8_sig')
# tabels_csv_df_cer_metas_cids_only.shape

## Load Figures and merge with regdocs metadata

In [395]:
#load figure data
fig_location = 'F:/Environmental Baseline Data/Version 4 - Final/Support files/final_figs.csv'
fig_df = pd.read_csv(fig_location, usecols= ['Name', 'DataID'])
fig_df.columns = ['Title', 'DataID']
fig_df['DataID'] = fig_df['DataID'].astype(str)
fig_df.head(2)

,Title,DataID
0,Figure 13.1-1 EnCana Ekwan Pipeline,268706
1,Figure 13.3-1 CEA Framework,268706


In [396]:
#merge with regdocs metadata
fig_df_meta = fig_df.merge(df_esa_with_gis_components_short_names, on = 'DataID', how = 'left')
fig_df_meta['Content type'] = 'Figure'
# fig_df_meta.head(2)
# fig_df_meta.columns

### Combine Figures and Tables

In [397]:
df_tables_figures = pd.concat([fig_df_meta, tabels_csv_df_cer_meta_no_cids], sort=False).reset_index(drop = True)
df_tables_figures.shape
df_tables_figures.tail(2)

(30082, 37)

,Title,DataID,file_name,application_name,esa_folder_link,DocumentID,esa_download_link,DataID_pdf,Section,Application filing date,...,Location,Commodity,Download folder name,Content type,csv_name,Page Number,Table Number,csv_path,shape,csv_download_link
30080,Table 2 Occurrences Of Species With Special Co...,895339,B2-32 - Volume IIA - ESA - Appendix 9 - Wildli...,2012-12-14 - Application for the Edmonton to H...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3E3A7,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,895339.pdf,Large Projects (over 40 km),2012-12-14,...,"Alberta, All",Oil,dmntnhrdst,Table,895339_14_1.csv,14,1,F:/Environmental Baseline Data/Version 4 - Fin...,"(7, 5)",http://dweb5/esa-ees/dmntnhrdst/895339_14_1.csv
30081,Table 3 Summary Of Potential Species With Spec...,895339,B2-32 - Volume IIA - ESA - Appendix 9 - Wildli...,2012-12-14 - Application for the Edmonton to H...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3E3A7,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,895339.pdf,Large Projects (over 40 km),2012-12-14,...,"Alberta, All",Oil,dmntnhrdst,Table,895339_23_1.csv,23,1,F:/Environmental Baseline Data/Version 4 - Fin...,"(14, 2)",http://dweb5/esa-ees/dmntnhrdst/895339_23_1.csv


In [398]:
df_tables_figures.to_csv('F:/Environmental Baseline Data/Version 4 - Final/Indices/Index 3 V4 CSV2 LITE.csv', encoding = 'utf_8_sig')

In [399]:
df_tables_figures.columns

Index(['Title', 'DataID', 'file_name', 'application_name', 'esa_folder_link',
       'DocumentID', 'esa_download_link', 'DataID_pdf', 'Section',
       'Application filing date', 'link to folder with ESA', 'Hearing order',
       'Folder link', 'Company Name', 'Consultant Name', 'Status in GIS',
       'Column1', 'RTS English Title = Project name',
       'Regulatory Instruments Issued', 'Pipeline Named in GIS',
       'Application title short', 'Link to Application', 'Link to Decision',
       'Component', 'Component PDF Name', 'Component Index', 'short_name',
       'Location', 'Commodity', 'Download folder name', 'Content type',
       'csv_name', 'Page Number', 'Table Number', 'csv_path', 'shape',
       'csv_download_link'],
      dtype='object')

## Create csv for ESA Data Bank dashboard FINAL (delete and rename some columns)

In [402]:
# 
esa_final_df = df_tables_figures[['Title', 'Content type', 'Application title short', 'short_name', 'Application filing date', 'Company Name',
                                 'Commodity', 'file_name', 'Component PDF Name', 'esa_folder_link', 'DocumentID', 'DataID', 
                                  'esa_download_link','Section', 'Location', 'Hearing order', 'Consultant Name', 'Column1' , 
                                  'Regulatory Instruments Issued', 'Link to Application', 'Link to Decision',
                                 'Component PDF Name', 'csv_download_link', 'Page Number']]

esa_final_df.rename(columns = {'Application title short' : 'Application Name', 'short_name' : 'Application Short Name', \
                              'file_name' : 'File Name', 'esa_folder_link' : 'ESA Folder URL', \
                               'esa_download_link' : 'PDF Download URL', 'Section' : 'Application Type (NEB Act)', \
                                'Column1' : 'Pipeline Status', 'Regulatory Instruments Issued' : 'Regulatory Instrument(s)', \
                              'Link to Application' : 'Applicatin URL', 'Link to Decision' : 'Decision URL', \
                              'Component PDF Name' : 'PDF Section(s)', 'csv_download_link' : 'Component download URL', \
                              'Page Number' : 'PDF Page#'}, inplace = True)

esa_final_df.to_csv('F:/Environmental Baseline Data/Version 4 - Final/Tableau files/Prototype/ESA Data Bank Final.csv', encoding = 'utf_8_sig')
